# Explore Obspy

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from math import radians, cos, sin, asin, sqrt
import os
import pandas as pd
import numpy as np
import geopandas as gpd
from obspy.clients.fdsn import Client
from obspy.core.utcdatetime import UTCDateTime
from shapely.geometry import Point, Polygon


import init
import whaletracks.common. constants as cn
from common_python.database import database_util as util
import obspy

## Constants

## ObsPy

In [3]:
def getStations(network_code, starttime= UTCDateTime("2001-01-01"), 
                endtime=UTCDateTime("2020-01-02")):
    """
    Provides a dataframe of station metadata.
    :param str network_code:
    :return pd.DataFrame:
        station_code, channel_count, start_time, end_time, access, latitude, longitude, elevation
    """
    KEYS = [
     'code',
     'creation_date',
     'elevation',
     'end_date',
     'latitude',
     'longitude',
     'start_date',
     'termination_date',
     'total_number_of_channels'
    ]
    client = Client("IRIS")
    inventory = client.get_stations(network=network_code, station="*",
        starttime=starttime, endtime=endtime)
    network = inventory[0]
    station_count = network.total_number_of_stations
    station_dct = {k: [] for k in KEYS}
    station_dct["network"] = list(np.repeat(network_code, station_count))
    for idx in range(station_count):
        station = network[idx]
        for key in KEYS:
            station_dct[key].append(station.__getattribute__(key))
    if False:
        station_lines = network.get_contents()["stations"]
        station_codes = []
        for line in station_lines:
            splits = line.split(" ")
            station_codes.append(splits[0][-4:])
        return station_codes
    return pd.DataFrame(station_dct)



station_df = getStations("7D")

In [ ]:

starttime = UTCDateTime("2001-01-01")

endtime = UTCDateTime("2020-01-02")
client = Client("IRIS")

inventory = client.get_stations(network="7D", station="*",

                                starttime=starttime,

                                endtime=endtime)
client = Client("IRIS")
#print(inventory)

if False:
    client = Client("IRIS")

    t1 = UTCDateTime("2010-02-27T06:30:00.000")

    t2 = t1 + 1

    t3 = t1 + 3

    bulk = [("IU", "ANMO", "*", "BHZ", t1, t2),

            ("IU", "AFI", "1?", "BHE", t1, t3),

            ("GR", "GRA1", "*", "BH*", t2, t3)]

    inv = client.get_stations_bulk(bulk, level="channel")
    print(inv)  

In [ ]:
inventory.get_contents()

In [ ]:
net = inventory[0]
net.total_number_of_stations

In [ ]:
net.stations

In [ ]:
station = net[0]; station

In [ ]:
str(station.creation_date)

In [ ]:
station.latitude, station.longitude

## Acquiring Channel Data

In [4]:
starttime = UTCDateTime("2001-01-01")
endtime = UTCDateTime("2020-01-02")
client = Client("IRIS")
inventory = client.get_stations(network="7D", station="*",
                                starttime=starttime,
                                endtime=endtime)

In [12]:
stations = [net[n] for n in range(net.total_number_of_stations) if len(net[n].channels) > 0]

In [16]:
net[2]

Station FN01C (LDEO OBS TRAWL-RESISTANT)
	Station Code: FN01C
	Channel Count: 0/9 (Selected/Total)
	2013-08-30T00:00:00.000000Z - 2014-06-27T23:59:59.000000Z
	Access: open 
	Latitude: 46.88, Longitude: -124.33, Elevation: -56.0 m
	Available Channels:


In [17]:
net

Network 7D (Cascadia Initiative Community Experiment - OBS Component)
	Station Count: 259/259 (Selected/Total)
	2011-01-01T00:00:00.000000Z - 2017-12-31T23:59:59.000000Z
	Access: open
	Contains:
		Stations (259):
			7D.FC03D (LDEO OBS TRAWL-RESISTANT)
			7D.FN01A (LDEO Trawl-Resistant OBS Site FN01A)
			7D.FN01C (LDEO OBS TRAWL-RESISTANT)
			7D.FN02C (LDEO OBS TRAWL-RESISTANT)
			7D.FN03A (LDEO Trawl-Resistant OBS Site FN03A)
			7D.FN03C (LDEO OBS TRAWL-RESISTANT)
			7D.FN04C (LDEO OBS TRAWL-RESISTANT)
			7D.FN05A (LDEO OBS TRAWL-RESISTANT)
			7D.FN05C (LDEO OBS TRAWL-RESISTANT)
			7D.FN06A (LDEO Trawl-Resistant OBS Site FN06A)
			7D.FN06C (LDEO OBS TRAWL-RESISTANT)
			7D.FN07A (LDEO Trawl-Resistant OBS Site FN07A)
			7D.FN07C (LDEO OBS TRAWL-RESISTANT)
			7D.FN08A (LDEO Trawl-Resistant OBS Site FN08A)
			7D.FN08C (LDEO OBS TRAWL-RESISTANT)
			7D.FN09A (LDEO Trawl-Resistant OBS Site FN09A)
			7D.FN09C (LDEO OBS TRAWL-RESISTANT)
			7D.FN10A (LDEO Trawl-Resistant OBS Site FN10A)
			7D.FN

In [25]:
UTCDateTime("2013-01-01") + 3600

2013-01-01T01:00:00.000000Z

In [26]:
starttime = UTCDateTime("2013-01-01")
endtime = starttime + 3600
st_raw = client.get_waveforms(network="7D", station="*", location = "*",
                              channel="BHZ,HHZ", starttime=starttime, endtime=endtime)

In [28]:
dir(st_raw)

['__add__',
 '__class__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getslice__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__mul__',
 '__ne__',
 '__new__',
 '__nonzero__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_cleanup',
 '_dummy_stream_from_string',
 '_get_common_channels_info',
 '_ltrim',
 '_merge_checks',
 '_repr_pretty_',
 '_rotate_specific_channels_to_zne',
 '_rotate_to_zne',
 '_rtrim',
 '_trim_common_channels',
 'append',
 'attach_response',
 'clear',
 'copy',
 'count',
 'cutout',
 'decimate',
 'detrend',
 'differentiate',
 'extend',
 'filter',
 'get_gaps',
 'insert',
 'integrate',
 'interpolate',
 'max',
 'merge',
 'normalize',
 'plot',
 'pop',
 'print_gaps',
 'remove',
 'remove_

In [29]:
# Can get channel gap information, but not other metadata?
st_raw._get_common_channels_info()

{('7D', 'G28B', '', 'BH?'): {'start': 2013-01-01T00:00:00.013307Z,
  'end': 2013-01-01T00:59:59.993307Z,
  'gaps': [],
  'channels': {'BHZ': {'start': 2013-01-01T00:00:00.013307Z,
    'end': 2013-01-01T00:59:59.993307Z}}},
 ('7D', 'J06B', '', 'BH?'): {'start': 2013-01-01T00:00:00.013900Z,
  'end': 2013-01-01T00:59:59.993900Z,
  'gaps': [],
  'channels': {'BHZ': {'start': 2013-01-01T00:00:00.013900Z,
    'end': 2013-01-01T00:59:59.993900Z}}},
 ('7D', 'M18B', '', 'HH?'): {'start': 2013-01-01T00:00:00.007700Z,
  'end': 2013-01-01T00:59:59.999700Z,
  'gaps': [],
  'channels': {'HHZ': {'start': 2013-01-01T00:00:00.007700Z,
    'end': 2013-01-01T00:59:59.999700Z}}},
 ('7D', 'FS20B', '', 'HH?'): {'start': 2013-01-01T00:00:00.000500Z,
  'end': 2013-01-01T00:59:59.992720Z,
  'gaps': [],
  'channels': {'HHZ': {'start': 2013-01-01T00:00:00.000500Z,
    'end': 2013-01-01T00:59:59.992720Z}}},
 ('7D', 'G20B', '', 'BH?'): {'start': 2013-01-01T00:00:00.010398Z,
  'end': 2013-01-01T00:59:59.990398Z,
  